In [1]:
# A multi classes image classifier, based on convolutional neural network using Keras and Tensorflow. 
# A multi-label classifier (having one fully-connected layer at the end), with multi-classification (18 classes, in this instance).
# Largely copied from the code https://gist.github.com/seixaslipe
# This is based on these posts: https://medium.com/alex-attia-blog/the-simpsons-character-recognition-using-keras-d8e1796eae36
# Data downloaded from Kaggle 
# Will emulate the image classification functionlities for Neuro Pathology images/slides (WSI-Whole Slide images)
# Will implement/include data manipulating functionalities based on Girder (https://girder.readthedocs.io/en/latest/)
# Has 6 convulsions, filtering start with 64, 128, 256 with flattening to 1024
# Used Keras.ImageDataGenerator for Training/Validation data augmentation and the augmented images are flown from respective directory
# Environment: A docker container having Keras, TensorFlow, Python-2 with GPU based execution

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import Callback
import datetime, time, os, sys
import numpy as np
import h5py
import matplotlib as plt
plt.use('Agg')
import matplotlib.pyplot as pyplot
pyplot.figure
import pickle 
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import subprocess
import pandas as pd

import nvidia_smi as nvs
import io
import pickle
import json

import glob


try:
    to_unicode = unicode
except NameError:
    to_unicode = str


Using TensorFlow backend.


In [ ]:
#!pip install nvidia-ml-py --user

In [32]:
modelInfo = {}
modelInfo['Device']  = {} ## Initialize an object to store info on the model and time info

nvs.nvmlInit()

driverVersion = nvs.nvmlSystemGetDriverVersion()
print("Driver Version: {}".format(driverVersion))
modelInfo['Device']['driverVersion']  = driverVersion

# e.g. will print:
#   Driver Version: 352.00
deviceCount = nvs.nvmlDeviceGetCount()
deviceNames = []
for i in range(deviceCount):
    handle = nvs.nvmlDeviceGetHandleByIndex(i)
    dvn = nvs.nvmlDeviceGetName(handle) # store the device name
    print("Device {}: {}".format(i,  dvn))
    deviceNames.append(dvn)
    # e.g. will print:
    #  Device 0 : Tesla K40c
nvs.nvmlShutdown()

modelInfo['Device']['deviceNames']  = deviceNames


### These parameters can be tuned and may affect classification results or accuracy
img_width, img_height = 64, 64
epochs = 50
batch_size = 64


modelInfo['batch_size'] = batch_size
modelInfo['epochs'] = epochs
modelInfo['img_width'] = 64
modelInfo['img_height'] = 64
 

### Define input dirs and output for results which contain the models as well as stats on the run
train_data_dir = '/data/train' 
validation_data_dir = '/data/test' 

resultsDir ="/app/results/"
if not os.path.isdir(resultsDir):
    os.makedirs(resultsDir)

nb_train_samples = 0

for root, dirs, files in os.walk(train_data_dir):
    nb_train_samples += len(files)

nb_validation_samples = 0
for root, dirs, files in os.walk(validation_data_dir):
    nb_validation_samples += len(files)


# Model definition
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip=True)

# Only rescaling for validation
valid_datagen = ImageDataGenerator(rescale=1. / 255.0)

# Flows the data directly from the directory structure, resizing where needed
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

NumLabels = len(validation_generator.class_indices)

'''
6-conv layers - added on 06/21, Raj
'''
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same')) 
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NumLabels, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# Captures GPU usage
#subprocess.Popen("timeout 120 nvidia-smi --query-gpu=utilization.gpu,utilization.memory --format=csv -l 1 | sed s/%//g > /app/results/GPU-stats.log",shell=True)



Driver Version: 384.130
Device 0: GeForce GTX 1050
Found 19548 images belonging to 20 classes.
Found 990 images belonging to 20 classes.


In [ ]:
# Timehistory callback to get epoch run times
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

# Model fitting and training run
simpsonsModel = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,

    callbacks=[time_callback])    


print "Finished running the basic model... trying to save results now.."


Epoch 1/50


In [15]:
# To write the each epoch run time into a json file
now = datetime.datetime.now()
filetime = str(now.year)+str(now.month)+str(now.day)+'_'+str(now.hour)+str(now.minute)

modelInfo['epochTimeInfo'] = time_callback.times


## Write out the h5/model
modelfilename=resultsDir+'Simpsonsmodel_'+filetime+'.h5'
model.save(modelfilename)

## This outputs the training and validation accuracy and loss functions for each epoch
## This will be graphed as well using plotly ... you can use this data to look for overfitting
## and/or when you can stop training your model because it stops improving
modelInfo['historyData'] =  pd.DataFrame(simpsonsModel.history).to_dict(orient='records')

###target_names maps the character names (or labels) to the index(integer) used in the output files
modelInfo['target_names']  = validation_generator.class_indices

modelInfo['labelname_to_index']  = validation_generator.class_indices
modelInfo['index_to_labelname']  = {(v,k) for k,v in validation_generator.class_indices.iteritems() }

In [18]:
target_names = validation_generator.class_indices

## Prediction for TEST data set
TRAIN_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
## This yields a probability for the TEST data set of the likelihood a given image is a member of a class

##Prediction for TEST data set... this is the best guess i.e. highest matching class/label
TRAIN_pred_label = np.argmax(TRAIN_pred, axis=1)

cnf_matrix = confusion_matrix(validation_generator.classes, TRAIN_pred_label)
cls_rpt = classification_report(validation_generator.classes, TRAIN_pred_label, target_names=target_names) 


In [12]:
print(cls_rpt)

                          precision    recall  f1-score   support

charles_montgomery_burns       0.15      0.10      0.12        48
            ned_flanders       0.06      0.06      0.06        50
           homer_simpson       0.00      0.00      0.00        50
           lenny_leonard       0.04      0.06      0.05        48
  abraham_grampa_simpson       0.12      0.14      0.13        50
            mayor_quimby       0.09      0.06      0.07        49
            chief_wiggum       0.02      0.02      0.02        50
          edna_krabappel       0.04      0.04      0.04        50
  apu_nahasapeemapetilon       0.02      0.02      0.02        50
       principal_skinner       0.07      0.08      0.07        50
           marge_simpson       0.12      0.02      0.03        50
             moe_szyslak       0.04      0.06      0.05        50
            nelson_muntz       0.08      0.08      0.08        50
        krusty_the_clown       0.05      0.02      0.03        50
         

In [17]:
print(cls_rpt)

                          precision    recall  f1-score   support

charles_montgomery_burns       0.07      0.06      0.07        48
            ned_flanders       0.07      0.06      0.06        50
           homer_simpson       0.13      0.10      0.11        50
           lenny_leonard       0.06      0.04      0.05        48
  abraham_grampa_simpson       0.08      0.12      0.10        50
            mayor_quimby       0.14      0.06      0.09        49
            chief_wiggum       0.14      0.08      0.10        50
          edna_krabappel       0.05      0.10      0.07        50
  apu_nahasapeemapetilon       0.02      0.02      0.02        50
       principal_skinner       0.02      0.02      0.02        50
           marge_simpson       0.05      0.04      0.05        50
             moe_szyslak       0.05      0.04      0.04        50
            nelson_muntz       0.02      0.02      0.02        50
        krusty_the_clown       0.00      0.00      0.00        50
         

In [6]:
print(cls_rpt)  ### This is a 20 by 20 matrix

## This looks cool, but we need to turn it into a table I guess?

                          precision    recall  f1-score   support

charles_montgomery_burns       0.05      0.04      0.05        48
            ned_flanders       0.10      0.10      0.10        50
           homer_simpson       0.05      0.06      0.06        50
           lenny_leonard       0.04      0.04      0.04        48
  abraham_grampa_simpson       0.09      0.10      0.09        50
            mayor_quimby       0.00      0.00      0.00        49
            chief_wiggum       0.03      0.02      0.02        50
          edna_krabappel       0.02      0.02      0.02        50
  apu_nahasapeemapetilon       0.06      0.06      0.06        50
       principal_skinner       0.06      0.06      0.06        50
           marge_simpson       0.04      0.04      0.04        50
             moe_szyslak       0.02      0.02      0.02        50
            nelson_muntz       0.10      0.10      0.10        50
        krusty_the_clown       0.06      0.06      0.06        50
         

In [19]:
## Turning into classification report into classification object
avgresults = cls_rpt.strip().split('\n')[-1].split()

In [20]:
overallResults={'label' : 'avg/total', 'precision': avgresults[3], 'recall':avgresults[4],'f1-score':avgresults[5], 'support':avgresults[6]}


In [24]:
precision, recall, fscore, support  =  precision_recall_fscore_support(validation_generator.classes, TEST_pred)

In [25]:
modelInfo['classificationObject'] =  characterResultsArray =  {
    'label': validation_generator.class_indices.keys(),
    'precision': precision,
    'recall':recall,
    'fscore': fscore, 'support':support,
    'overallResults':{'label' : 'avg/total', 
                      'precision': avgresults[3], 
                      'recall':avgresults[4],
                      'f1-score':avgresults[5],
                      'support':avgresults[6]}}

In [ ]:
modelInfo['classificationObject']

In [27]:
modelInfo['confusion_matrix'] = confusion_matrix(validation_generator.classes, TEST_pred)

In [ ]:
modelInfo['confusion_matrix']



In [ ]:
###   filename --- CLASS


### LAST BUT NOT LEAST --- 



# MAKE IT A PARAMETER OUTPUT MODELPREDICTIOJ FOR TRAIN AND TEST OR JUST TEST  

# for image in glob.glob('/data/train/*/'):

#     I WANT
    
#     ['filename': "somename", 'actualImageLabel': asIndex, 'modelPrection': X ]
    
   


In [28]:
import glob
import os

dgWant = []

model=load_model(modelfilename)

for fld in os.listdir('/data/test/'): 
    trueLabel = fld
    for img in os.listdir('/data/test/%s/' %trueLabel): 
        imgPath = "/data/test/%s/%s" % (fld, img)
        x = image.load_img(imgPath, target_size=(64,64))
        x = image.img_to_array(x)
        x = x.reshape((1,) + x.shape)
        x = x/255.
        pr=model.predict(x)
        curr = {'filename': img, 'actualImageLabel': fld, 'modelprediction':pr} 
        dgWant.append(curr)

In [29]:
d = dgWant[300]['modelprediction']
maxIndex = np.argmax(d)

dict(modelInfo['index_to_labelname'])[maxIndex]

'homer_simpson'

In [ ]:
dict(modelInfo['index_to_labelname'])

In [31]:
dgWant[3]

{'actualImageLabel': 'chief_wiggum',
 'filename': 'chief_wiggum_27.jpg',
 'modelprediction': array([[4.29841629e-09, 1.28591315e-09, 1.29947765e-07, 2.76901956e-05,
         9.99919176e-01, 2.25431722e-06, 1.29766701e-11, 5.24551024e-06,
         8.81407658e-09, 4.40973110e-07, 4.12774710e-07, 1.00335065e-07,
         1.16415742e-06, 3.22805960e-07, 8.68021687e-07, 4.18807031e-05,
         1.47505763e-09, 1.46813672e-08, 2.85744591e-07, 3.95747257e-08]],
       dtype=float32)}